In [1]:
import pydgraph
import json

In [2]:
client_stub = pydgraph.DgraphClientStub('localhost:9080')
client = pydgraph.DgraphClient(client_stub)

In [10]:
# Drop All - discard all data and start from a clean slate.
def drop_all(client):
    return client.alter(pydgraph.Operation(drop_all=True))

# Set schema.
def set_schema(client):
    schema = """
    name: string @index(exact) .
    depends: [uid] @reverse .
    version: string .
    type Package {
        name
        depends
        version
    }
    """
    return client.alter(pydgraph.Operation(schema=schema))

drop_all(client)

set_schema(client)



In [28]:
txn = client.txn()

try:
    django = {
        'uid': '_:django',
        'dgraph.type': 'Package',
        'name': 'Django',
        'depends': [
            {
                'uid': '_:pytz',
                'dgraph.type': 'Package',
                'name': 'pytz',
                'version_specifier': '*'
            },
            {
                'uid': '_:sqlparse',
                'dgraph.type': 'Package',
                'name': 'sqlparse',
                'version_specifier': '>=0.2.2'
            },
            {
                'uid': '_:asgiref',
                'dgraph.type': 'Package',
                'name': 'asgiref',
                'version_specifier': '~=3.2'
            },
                        {
                'uid': '_:argon2-cffi',
                'dgraph.type': 'Package',
                'name': 'argon2-cffi',
                'version_specifier': '>=16.1.0'
            },
                        {
                'uid': '_:bcrypt',
                'dgraph.type': 'Package',
                'name': 'bcrypt',
                'version_specifier': '*'
            },
        ],
    }

    asgiref = {
        'uid': 'asgiref',
        'dgraph.type': 'Package',
        'name': 'asgiref',
        'depends': [
            {
                'uid': 'pytest',
                'dgraph.type': 'Package',
                'name': 'pytest',
                'version_specifier': '~=4.3.0'
            },
            {
                'uid': '_:pytest-asyncio',
                'dgraph.type': 'Package',
                'name': 'pytest-asyncio',
                'version_specifier': '~=0.10.0'
            },
           
        ],
    }
    
    pytest = {
        'uid': 'pytest',
        'dgraph.type': 'Package',
        'name': 'pytest',
        'depends': [
            {
                'uid': 'py',
                'dgraph.type': 'Package',
                'name': 'py',
                'version_specifier': '>=1.5.0'
            },
            {
                'uid': 'packaging',
                'dgraph.type': 'Package',
                'name': 'packaging',
                'version_specifier': '*'
            },
           
        ],
    }
    
    response = txn.mutate(set_obj=django)
    print(response)
    #response = txn.mutate(set_obj=asgiref)
    #response = txn.mutate(set_obj=pytest)

    txn.commit()
finally:
  txn.discard()

txn {
  start_ts: 10019
  keys: "15x98ilja01i"
  keys: "1cagssnvzzdxl"
  keys: "1jce8w6ahf3mn"
  keys: "1pbw4rrumo7hm"
  keys: "1qlu7kry98rze"
  keys: "1yhcp8x88nrft"
  keys: "220085ozcopbe"
  keys: "24fht1azcqn7m"
  keys: "27w0ya0gw4799"
  keys: "2a6oy5qo9hlmn"
  keys: "2f48ij1allqaz"
  keys: "2f4ib30alug5c"
  keys: "2h1qfumd5xvxk"
  keys: "2h1qfumd5xvxl"
  keys: "2h1qfumd5xvxn"
  keys: "2h1qfumd5xvxo"
  keys: "2h1qfumd5xvxz"
  keys: "2v9dtuhid9nju"
  keys: "2ywevowb30s58"
  keys: "3e8l1s015dktm"
  keys: "3kml8m6z0epfh"
  keys: "3kp30f4952b01"
  keys: "3pz3l3uyb64a6"
  keys: "3t4j778qs7eki"
  keys: "3t4j778qs7ekk"
  keys: "3t4j778qs7ekl"
  keys: "3t4j778qs7ekm"
  keys: "3t4j778qs7ekn"
  keys: "3t4j778qs7el5"
  keys: "3tyll093pssbi"
  keys: "6zaqm3fvfcsi"
  keys: "i7lvspl71ke2"
  keys: "j6j8ldkmeqh8"
  keys: "smcdrrb8bfn7"
  preds: "1-depends"
  preds: "1-dgraph.type"
  preds: "1-name"
  preds: "1-version_specifier"
}
latency {
  parsing_ns: 80098
  processing_ns: 32267416
  assign_tim

In [19]:
# Query for data.
def query_package(client, package_name):
    # Run query.
    query = """query all($a: string) {
        all(func: eq(name, $a)) {
            uid
            name
            version
            depends {
                name
                version
                version_specifier
            }
        }
    }"""

    variables = {'$a': package_name}
    res = client.txn(read_only=True).query(query, variables=variables)
    packages = json.loads(res.json)

    # Print results.
    return packages


In [20]:

django_dgraph = query_package(client, 'Django')


In [22]:
django_dgraph['all'][0]['uid']

'0x272e'

In [23]:
txn = client.txn()

try:
    django = {
        'uid': django_dgraph['all'][0]['uid'],
        'dgraph.type': 'Package',
        'name': 'Django',

    }

    
    response = txn.mutate(set_obj=django)
    #response = txn.mutate(set_obj=asgiref)
    #response = txn.mutate(set_obj=pytest)

    txn.commit()
finally:
  txn.discard()

In [26]:
django_dgraph = query_package(client, 'Django2')


In [27]:
django_dgraph

{'all': [{'uid': '0x272e',
   'name': 'Django2',
   'depends': [{'name': 'pytz', 'version_specifier': '*'},
    {'name': 'sqlparse', 'version_specifier': '>=0.2.2'},
    {'name': 'asgiref', 'version_specifier': '~=3.2'},
    {'name': 'argon2-cffi', 'version_specifier': '>=16.1.0'},
    {'name': 'bcrypt', 'version_specifier': '*'}]}]}